In [1]:
from data_access.twitter_reader import TwitterReader

In [7]:
tr = TwitterReader('./local_configs/api_keys.json')
query_dict = {'#': ['Roma', 'Milano'], 'keyword': ['neve'], 'max_results':10}
tr.set_query(query_dict)
res  =tr.read()

https://api.twitter.com/2/tweets/search/recent?query=%28%23Roma+OR+%23Milano%29neve&max_results=100&tweet.fields=author_id,text,created_at
200


In [8]:
res['meta']

{'newest_id': '1335860779082522624',
 'oldest_id': '1334204472390787073',
 'result_count': 100,
 'next_token': 'b26v89c19zqg8o3fosetlg0yv433cw6t4mepyc5h2qbnh'}

In [9]:
save_tweets(res['data'])

Table exists.


In [10]:
len(get_tweets())

134

In [ ]:
import pandas as pd
import nltk

nltk.download('punkt')

In [ ]:
df = pd.read_csv('./data/tweets.csv')

In [ ]:
df = df['text']
df.to_csv(r'./data/text_only.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

In [ ]:
reader = PlaintextCorpusReader('./data/', '.*\.txt')

In [ ]:
reader.sents('text_only.txt')[1]